In [1]:
# !pip install pycapnp
# !pip install typer
# !pip install ipywidgets

In [2]:
import asyncio
import zmq.asyncio

import os
import numpy as np
import logging
from pathlib import Path

import capnp
import typer
import zmq.asyncio
import time
from deepdrrzmq.utils.zmq_util import zmq_poll_latest

from deepdrrzmq.utils.typer_util import unwrap_typer_param
from deepdrrzmq.utils.server_util import make_response, DeepDRRServerException, messages
import random
import string

import threading
from IPython.display import display
import ipywidgets as widgets
import time
from threading import Event
import asyncio

kj/filesystem-disk-unix.c++:1703: warning: PWD environment variable doesn't match current directory; pwd = /home/wangl


In [3]:
rep_port=40100
pub_port=40101
sub_port=40102

In [4]:
context = zmq.asyncio.Context()
# context = zmq.Context()
context.__enter__()

sub_socket = context.socket(zmq.SUB)
sub_socket.hwm = 10000

pub_socket = context.socket(zmq.PUB)
pub_socket.hwm = 10000

# addr = "192.158.1.54"
addr = "localhost"
pub_socket.connect(f"tcp://{addr}:{pub_port}") # make variable
sub_socket.connect(f"tcp://{addr}:{sub_port}")
# pub_socket.connect(f"tcp:10.0.0.7") # make variable
# sub_socket.connect(f"tcp:10.0.0.7")

sub_socket.subscribe(b"")
# sub_socket.subscribe(b"/replayd/")

In [6]:
class ReplayControllerInterface:
    def __init__(self):
        self.record_button = widgets.Button(description='New Session', disabled=True, button_style='', icon='circle')
        self.stop_button = widgets.Button(description='Stop', disabled=True, button_style='', icon='square')
        self.session_id = widgets.Text(value='None', placeholder='', description='Session ID:', disabled=True)
        self.record_status = widgets.Text(value='None', placeholder='', description='Recording:', disabled=True)
        self.log_dropdown = widgets.Dropdown(
            options=['1', '2', '3'],
            value='2',
            description='Number:',
            disabled=False,
        )
        self.change_log_button = widgets.Button(description='Change Log', disabled=False, button_style='', icon='square')
        self.log_dropdown_init = False
        self.last_heard_time = 0

    async def start(self):
        await asyncio.gather(
            self.publish(),
            self.listen(),
            self.ui_update(),
        )

    async def publish(self):
        while True:
            if not self.log_dropdown_init:
                pub_socket.send_multipart([b"/replayd/in/listrequest/", b""])

            if time.time() - self.last_heard_time > 3:
                self.session_id.value = "Disconnected"
                self.record_status.value = "Disconnected"
                self.record_button.disabled = True
                self.stop_button.disabled = True
                self.log_dropdown.disabled = True
            
            await asyncio.sleep(1)

    async def listen(self):

        while True:
            latest_msgs = await zmq_poll_latest(sub_socket)

            for topic, data in latest_msgs.items():
                if topic == b"/replayd/status/":
                    self.last_heard_time = time.time()
                    self.record_button.disabled = False
                    self.stop_button.disabled = False
                    self.log_dropdown.disabled = False
                    with messages.ReplayerStatus.from_bytes(data) as msg:
                        pass
                        # session_id.value = msg.sessionId if msg.recording else "no session"
                        # record_status.value = str(msg.recording)
                        # record_button.button_style = 'danger' if msg.recording else ''
                elif topic == b"/replayd/list/":
                    with messages.LogList.from_bytes(data) as msg:
                        log_list = [x.id for x in msg.logs]
                        self.log_dropdown.options = reversed(log_list)
                        self.log_dropdown_init = True 

            await asyncio.sleep(1)

    def on_record_button_clicked(self, b):
        pub_socket.send_multipart([b"/replayd/in/start/", b""])
        print("sent start")
        

    def on_stop_button_clicked(self, b):
        pub_socket.send_multipart([b"/replayd/in/stop/", b""])

    def on_selected_log_changed(self, b):
        print("selected log changed")
        msg = messages.LoadLogRequest.new_message()
        msg.logId = self.log_dropdown.value
        msg.autoplay = True
        msg.startTime = 0
        msg.loop = True
        pub_socket.send_multipart([b"/replayd/in/load/", msg.to_bytes()])

    async def ui_update(self):
        self.record_button.on_click(self.on_record_button_clicked)
        self.stop_button.on_click(self.on_stop_button_clicked)
        self.change_log_button.on_click(self.on_selected_log_changed)
        # display(widgets.HBox([record_button, stop_button, log_dropdown, change_log_button, record_status]))
        display(widgets.HBox([
            self.record_button,
            self.stop_button,
            self.log_dropdown,
            self.change_log_button,
            self.record_status
        ]))

# await start()
interface = ReplayControllerInterface()
await interface.start()